In [63]:
import altair as alt
import pandas as pd
from helpers import RAW_DATA, PROCESSED_DATA, find_files, FIGURES

# Table of contents
1. [Introduction](#introduction)
2. [ETL](#etl)
    1. [Extract data](#load_data)
        1. [fetch files](#fetch_files)
        2. [load files](#load_files)
        3. [metadata](#metadata)
    2. [Transform data](#transform)
        1. [deduplicate `ModalityDSC`](#clean_up)
    3. [Load data](#save_data)
        1. [Confirm save](#load_save)
3. [Analysis](#analysis)
    1. [Modality by Clinic versus Number of Visits](#graph_data)
    2. [Age distribution by Clinic](#age_distribution)
    3. [Age distribution by Modality](#age_by_modality)
    4. [Risk distribution by Modality](#modality_risk)
 

## Introduction <a name="introduction"></a>
The [GitHub page](https://github.com/caheredia/medical_data_analysis) contains the code necessary to reproduce this notebook. The plots can be downloaded [here]() as well. 

## ETL <a name="etl"></a>

### Extract <a name="load_data"></a>

#### fetch files <a name="fetch_files"></a>

In [21]:
files = find_files(RAW_DATA)

#### load files <a name="load_files"></a>

In [22]:
df = pd.DataFrame()
for file in files:
    temp_df = pd.read_csv(file)
    df = pd.concat([df, temp_df], ignore_index=True)
df.head()

,PatientID,AgeNBR,GenderCD,RelationshipDSC,ClinicID,ModalityDSC,VisitsCNT,RiskNBR
0,42534,39,F,Employee,A,Primary,1,NaN
1,169233,46,F,Dependent,A,Nurse Visit,3,0.165685
2,42880,59,F,Employee,B,|||Preventive,1,1.133132
3,118044,36,F,Employee,B,Health Coach,1,0.701008
4,36774,39,F,Dependent,A,Massage,1,1.221869


#### dataset metadata<a name="metadata"></a>

In [23]:
df.describe()

,PatientID,AgeNBR,VisitsCNT,RiskNBR
count,7273.000000,7273.000000,7273.000000,6937.000000
mean,102272.129520,58.753609,2.374123,0.989636
std,58336.218918,23.743300,3.599425,0.988562
min,85.000000,18.000000,1.000000,0.000072
25%,51925.000000,38.000000,1.000000,0.285526
50%,103086.000000,59.000000,1.000000,0.682753
75%,152914.000000,79.000000,2.000000,1.357584
max,202870.000000,99.000000,74.000000,9.312341


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7273 entries, 0 to 7272
Data columns (total 8 columns):
PatientID          7273 non-null int64
AgeNBR             7273 non-null int64
GenderCD           6688 non-null object
RelationshipDSC    7273 non-null object
ClinicID           7273 non-null object
ModalityDSC        7273 non-null object
VisitsCNT          7273 non-null int64
RiskNBR            6937 non-null float64
dtypes: float64(1), int64(3), object(4)
memory usage: 454.7+ KB


### Transform<a name="transform"></a>

#### clean up values in `ModalityDSC` column by removing `|||` chars <a name="clean_up"></a>

In [25]:
df.ModalityDSC = df.ModalityDSC.apply(lambda x: x.replace("|||", ""))

In [26]:
categorical_columns = ["GenderCD", "RelationshipDSC", "ClinicID", "ModalityDSC"]

for column in categorical_columns:
    # set columns as category type
    df[column] = df[column].astype("category")
    unique_values = df[column].unique()
    print(f"{column} unique values: ")
    for value in unique_values:
        print(f"\t\t\t\t {value}")

GenderCD unique values: 
				 F
				 M
				 nan
RelationshipDSC unique values: 
				 Employee
				 Dependent
ClinicID unique values: 
				 A
				 B
				 C
ModalityDSC unique values: 
				 Primary
				 Nurse Visit
				 Preventive
				 Health Coach
				 Massage
				 Lab Visit
				 Optometry
				 Chiropractic
				 Acupuncture
				 Physical Therapy
				 Dental
				 Nurse Practitioner
				 Dermatology
				 Mental Health
				 Screening
				 Bod Pod
				 Psychiatry


### Load data <a name="save_data"></a>

In [27]:
df.to_pickle(PROCESSED_DATA + "cleaned_up_health.pkl.gzip", compression="gzip")

#### Confirm save by loading transformed data <a name="load_save"></a>

In [41]:
files = find_files(PROCESSED_DATA, file_type=".gzip")
# load files
df = pd.DataFrame()
for file in files:
    temp_df = pd.read_pickle(file, compression="gzip")
    df = pd.concat([df, temp_df], ignore_index=True)
df.head()

,PatientID,AgeNBR,GenderCD,RelationshipDSC,ClinicID,ModalityDSC,VisitsCNT,RiskNBR
0,42534,39,F,Employee,A,Primary,1,NaN
1,169233,46,F,Dependent,A,Nurse Visit,3,0.165685
2,42880,59,F,Employee,B,Preventive,1,1.133132
3,118044,36,F,Employee,B,Health Coach,1,0.701008
4,36774,39,F,Dependent,A,Massage,1,1.221869


## Analysis <a name="analysis"></a>

### Modality by Clinic versus Number of Visits <a name="graph_data"></a>

In [42]:
df_modality_group = df.groupby(['ModalityDSC', 'ClinicID'])[['VisitsCNT']].sum().reset_index()
df_modality_group.head()

,ModalityDSC,ClinicID,VisitsCNT
0,Acupuncture,A,411.0
1,Acupuncture,B,421.0
2,Acupuncture,C,408.0
3,Bod Pod,A,50.0
4,Bod Pod,B,NaN


In [47]:
visits = (
    alt.Chart(df_modality_group)
    .mark_bar()
    .encode(
        x="sum(VisitsCNT):Q",
        y=alt.Y("ModalityDSC:N", sort="-x"),
        color="ClinicID:N",
        tooltip=["ClinicID", "sum(VisitsCNT):Q"],
    )
    .properties(width=700, height=500, title='Modality versus Visits [counts]',)
)
visits

alt.Chart(...)

#### save chart <a name="save_modality"></a>

In [99]:
visits.save(FIGURES + '/visits.html')

### Age distribution by Clinic <a name="age_distribution"></a>

In [100]:
age_dist = (
    alt.Chart(df[0:5000])
    .mark_area()
    .encode(
        x="AgeNBR:O",
        y=alt.Y(
            "sum(VisitsCNT):Q", title="Number of Visits", axis=alt.Axis(format="~s")
        ),
        facet=alt.Facet("ClinicID:N", columns=3),
        tooltip=["AgeNBR:O", "sum(VisitsCNT):Q"],
    ).interactive()
    .properties(title="Number of visitors by Age and Clinic", width=250, height=100)
)
age_dist

alt.Chart(...)

#### save chart <a name="save_age"></a>

In [101]:
age_dist.save(FIGURES + '/age_dist.html')

### Age by Modality <a name="age_by_modality"></a>

In [89]:
age_mod = (
    alt.Chart(df[0:5000])
    .transform_density(
        "AgeNBR", groupby=["ModalityDSC"], as_=["AgeNBR", "density"], extent=[0, 99],
    )
    .mark_area()
    .encode(x="AgeNBR:O", y="density:Q",)
    .properties(title="Age Distribution per modality", width=200, height=80)
    .facet("ModalityDSC:N", columns=3)
)
age_mod

alt.FacetChart(...)

#### save chart <a name="save_age_mod"></a>

In [95]:
age_mod.save(FIGURES + '/age_density_by_modality.html')

### Risk by Modality<a name="modality_risk"></a>

In [90]:
risk = (
    alt.Chart(df[0:5000])
    .transform_density(
        "RiskNBR", groupby=["ModalityDSC"], as_=["RiskNBR", "density"], extent=[0, 8],
    )
    .mark_area()
    .encode(x="RiskNBR:Q", y="density:Q",)
    .properties(title="Number of visitors by Age and Clinic", width=200, height=50)
    .facet("ModalityDSC:N", columns=3)
)
risk

alt.FacetChart(...)

#### save chart <a name="save_risk"></a>

In [97]:
risk.save(FIGURES + '/risk_by_modality.html')